<a href="https://colab.research.google.com/github/adityamishra5050/Coronavirus-tweets-NLP---Text-Classification-/blob/main/NLU_ass_1_task4(d).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bi-LSTM model**

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
Corona_NLP_train=pd.read_csv("/content/drive/MyDrive/Corona_NLP_train.csv",encoding='latin1')

In [ ]:
Corona_NLP_train['Location'].unique()

array(['London', 'UK', 'Vagabonds', ..., 'Juba south sudan', 'OHIO',
       'i love you so much || he/him'], dtype=object)

In [ ]:
a=Corona_NLP_train['Sentiment'].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
y=Corona_NLP_train['Sentiment'].copy()

In [ ]:
Corona_NLP_train['Sentiment']=Corona_NLP_train['Sentiment'].map({'Positive':0,'Negative':1,'Neutral':2,'Extremely Positive':3,
                                           'Extremely Negative':4})

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import re

In [ ]:
def cleaner(tweet):

    # remove urls
    tweet = re.sub(r'http\S+', ' ', tweet)

    # remove html tags
    tweet = re.sub(r'<.*?>',' ', tweet)

    # remove digits
    tweet = re.sub(r'\d+',' ', tweet)

    # remove hashtags
    tweet = re.sub(r'#\w+',' ', tweet)

    # remove mentions
    tweet = re.sub(r'@\w+',' ', tweet)

    #removing stop words
    tweet = tweet.split()
    tweet = " ".join([word for word in tweet if not word in stop_words])

    return  tweet

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = stopwords.words('english')
Corona_NLP_train['OriginalTweet']=Corona_NLP_train['OriginalTweet'].apply(lambda x:x.lower())
Corona_NLP_train['OriginalTweet'] = Corona_NLP_train['OriginalTweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
train_cleaned = Corona_NLP_train['OriginalTweet'].apply(cleaner)
train_cleaned.head()

0                                                     
1    advice talk neighbours family exchange phone n...
2    coronavirus australia: woolworths give elderly...
3    food stock empty... please, panic, enough food...
4    ready supermarket outbreak. paranoid, food sto...
Name: OriginalTweet, dtype: object

In [ ]:
Corona_NLP_train['OriginalTweet']=train_cleaned

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.layers as Layers
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns


import numpy as np
import pandas as pd

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_cleaned)

X = tokenizer.texts_to_sequences(train_cleaned)

vocab_size = len(tokenizer.word_index)+1



print("Vocabulary size: {}".format(vocab_size))
print("\nExample:\n")
print("Sentence:\n{}".format(train_cleaned[6]))
print("\nAfter tokenizing :\n{}".format(X[6]))

X = pad_sequences(X, padding='post')
print("\nAfter padding :\n{}".format(X[6]))

Vocabulary size: 34757

Example:

Sentence:
cashier grocery store sharing insights prove credibility commented "i'm civics class know talking about".

After tokenizing :
[1023, 6, 3, 1116, 637, 2697, 9271, 9272, 3365, 17401, 1330, 42, 773, 2065]

After padding :
[ 1023     6     3  1116   637  2697  9271  9272  3365 17401  1330    42
   773  2065     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]


In [ ]:
replace_lable = {'Extremely Negative': 0,
            'Negative': 0,
            'Neutral': 1,
            'Positive':2,
            'Extremely Positive': 2
           }

labels = ['Negative', 'Neutral', 'Positive']

y.replace(replace_lable, inplace=True)

In [ ]:
tf.keras.backend.clear_session()

# hyper parameters
EPOCHS = 2
BATCH_SIZE = 32
embed_dim = 16
units = 256

model_bi_LSTM = tf.keras.Sequential([
    Layers.Embedding(vocab_size, embed_dim, input_length=X.shape[1]),
    Layers.Bidirectional(Layers.LSTM(units,return_sequences=True)),
    Layers.GlobalMaxPool1D(),
    Layers.Dropout(0.4),
    Layers.Dense(64, activation="relu"),
    Layers.Dropout(0.4),
    Layers.Dense(3)
])


model_bi_LSTM.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',metrics=['accuracy']
             )

model_bi_LSTM.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 16)            556112    
                                                                 
 bidirectional (Bidirectiona  (None, 40, 512)          559104    
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 512)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0

In [ ]:
history1 = model_bi_LSTM.fit(X, y, epochs=EPOCHS, validation_split=0.12, batch_size=BATCH_SIZE)

Epoch 1/2
1132/1132 [==============================] - 397s 345ms/step - loss: 0.7317 - accuracy: 0.6723 - val_loss: 0.4639 - val_accuracy: 0.8364
Epoch 2/2
1132/1132 [==============================] - 409s 362ms/step - loss: 0.3887 - accuracy: 0.8705 - val_loss: 0.4114 - val_accuracy: 0.8564


In [ ]:
fig = px.line(
    history1.history, y=['accuracy', 'val_accuracy'],
    labels={'index': 'epoch', 'value': 'accuracy'},width=800, height=400
)

fig.show()

In [ ]:
fig = px.line(
    history1.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'loss'},width=800, height=400
)

fig.show()

In [ ]:
Corona_NLP_test=pd.read_csv("/content/drive/MyDrive/Corona_NLP_test.csv")

In [ ]:
corona_X_test = Corona_NLP_test['OriginalTweet'].copy()
corona_y_test = Corona_NLP_test['Sentiment'].copy()

corona_X_test = corona_X_test.apply(cleaner)

corona_X_test = tokenizer.texts_to_sequences(corona_X_test)

corona_X_test = pad_sequences(corona_X_test, padding='post')

corona_y_test.replace(replace_lable, inplace=True)

In [ ]:
model_bi_LSTM.evaluate(corona_X_test,corona_y_test)

119/119 [==============================] - 12s 85ms/step - loss: 0.4609 - accuracy: 0.8328


[0.46088701486587524, 0.8328067660331726]